In [54]:
import numpy as np
import random
import math
from typing import Tuple, Literal, Union, Optional, List, Dict, NamedTuple, Callable, Any, Set
from queue import Queue
import warnings
import sys
import json
import time
import types
import pickle
import plotly.graph_objs as go
import plotly.express as px
import pandas as pd
import os

In [55]:
pd.set_option('display.max_rows', None)     # 모든 행 표시
pd.set_option('display.max_columns', None)  # 모든 열 표시
pd.set_option('display.width', None)        # 가로 폭 제한 없음
pd.set_option('display.max_colwidth', None) # 셀 내용 잘림 없이 표시

In [ ]:
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
pd.reset_option('display.width')
pd.reset_option('display.max_colwidth')

In [56]:
class Map(NamedTuple):
    grid: np.ndarray
    start: Union[Tuple[float, float], Tuple[float, float, float]]
    goal: Union[Tuple[float, float], Tuple[float, float, float]]
    obstacles: List[Union[Tuple[float, float, float, float], Tuple[float, float, float, float, float, float]]] # x, y, width, height or x, y, z, width, height, dimension
    size: Union[Tuple[int, int], Tuple[int, int, int]]

class PlannerResult(NamedTuple):
    success: bool
    path: List[Union[Tuple[float, float], Tuple[float, float, float]]]
    nodes: List[Union[Tuple[float, float], Tuple[float, float, float]]]
    edges: List[Tuple[Tuple[float, ...], Tuple[float, ...]]]  # (parent, child)

class Node:
    def __init__(self, position, parent=None, cost=0.0):
        self.position = position
        self.parent = parent
        self.cost = cost
        self.children = []
        self.valid = True  # 장애물 충돌 여부 등


In [ ]:
class MapGenerator:
    def __init__(
        self,
        map_type: Literal["random", "multi_narrow", "maze"] = "random",
        map_size: Union[Tuple[int, int], Tuple[int, int, int]] = (50, 50),
        obstacle_percent: float = 0.2,
        min_obstacle_size: Union[Tuple[int, int], Tuple[int, int, int]] = (2, 2),
        max_obstacle_size: Union[Tuple[int, int], Tuple[int, int, int]] = (5, 5),
        max_obstacle_count: Optional[int] = None
    ):
        self.map_type = map_type
        self.map_size = map_size
        self.obstacle_percent = obstacle_percent
        self.obstacle_list: List[Tuple[int, ...]] = []
        self.min_size = min_obstacle_size
        self.max_size = max_obstacle_size
        self.max_count = max_obstacle_count
        self.is_3d = len(map_size) == 3
        self.map = self._init_map()
        self.start = None
        self.goal = None

    def _init_map(self):
        shape = self.map_size[::-1] if self.is_3d else (self.map_size[1], self.map_size[0])
        return np.zeros(shape, dtype=np.uint8)

    def generate(self, start: Tuple[int, ...], goal: Tuple[int, ...]) -> Dict[str, Union[np.ndarray, Tuple[int, ...], List[Tuple[int, ...]], Tuple[int, int]]]:
        self.start = start
        self.goal = goal
        tries = 0
        max_tries = 100

        while tries < max_tries:
            self.map = self._init_map()
            self.obstacle_list.clear()

            if self.map_type == "maze":
                self.map = self._generate_maze()
            elif self.map_type == "random":
                self._generate_random_obstacles(start, goal)
            elif self.map_type == "multi_narrow":
                self._generate_multi_narrow(start, goal)

            if self._path_exists(start, goal):
                return Map(
                    grid=self.map,
                    start=start,
                    goal=goal,
                    obstacles=self.obstacle_list,
                    size=self.map_size
                )
            tries += 1

        raise RuntimeError("Failed to generate a connected map after multiple attempts.")

    def _add_obstacle(self, coords: Tuple[int, ...]):
        self.obstacle_list.append(coords)

    def _path_exists(self, start, goal):
        visited = set()
        q = Queue()
        q.put(start)
        visited.add(start)

        dims = len(start)
        neighbors = [(-1,0), (1,0), (0,-1), (0,1)] if dims == 2 else \
                    [(-1,0,0),(1,0,0),(0,-1,0),(0,1,0),(0,0,-1),(0,0,1)]

        while not q.empty():
            node = q.get()
            if node == goal:
                return True

            for delta in neighbors:
                neighbor = tuple(node[i] + delta[i] for i in range(dims))
                if self._in_bounds(neighbor) and neighbor not in visited:
                    if self._is_free(neighbor):
                        visited.add(neighbor)
                        q.put(neighbor)
        return False

    def _in_bounds(self, p):
        if self.is_3d:
            z, y, x = p
            return 0 <= z < self.map.shape[0] and 0 <= y < self.map.shape[1] and 0 <= x < self.map.shape[2]
        else:
            y, x = p
            return 0 <= y < self.map.shape[0] and 0 <= x < self.map.shape[1]

    def _is_free(self, p):
        if self.is_3d:
            z, y, x = p
            return self.map[z, y, x] == 0
        else:
            y, x = p
            return self.map[y, x] == 0

    def _is_inside(self, point, x, y, z, ow, oh, od):
        px, py, *pz = point
        if z is None:
            return x <= px < x + ow and y <= py < y + oh
        else:
            pz = pz[0]
            return x <= px < x + ow and y <= py < y + oh and z <= pz < z + od

    def _generate_random_obstacles(self, start, goal):
        if self.is_3d:
            w, h, d = self.map_size
            total_voxels = w * h * d
            max_obs = int(total_voxels * self.obstacle_percent)
            count = 0
            for _ in range(10000):
                ow = random.randint(self.min_size[0], self.max_size[0])
                oh = random.randint(self.min_size[1], self.max_size[1])
                od = random.randint(self.min_size[2], self.max_size[2])
                x = random.randint(0, w - ow - 1)
                y = random.randint(0, h - oh - 1)
                z = random.randint(0, d - od - 1)
                if np.any(self.map[z:z+od, y:y+oh, x:x+ow]):
                    continue
                if self._is_inside(start, x, y, z, ow, oh, od) or self._is_inside(goal, x, y, z, ow, oh, od):
                    continue
                self.map[z:z+od, y:y+oh, x:x+ow] = 1
                self._add_obstacle((x, y, z, ow, oh, od))
                count += ow * oh * od
                if self.max_count and count >= self.max_count:
                    break
                if count >= max_obs:
                    break
        else:
            w, h = self.map_size
            total_cells = w * h
            max_obs = int(total_cells * self.obstacle_percent)
            count = 0
            for _ in range(10000):
                ow = random.randint(self.min_size[0], self.max_size[0])
                oh = random.randint(self.min_size[1], self.max_size[1])
                x = random.randint(0, w - ow - 1)
                y = random.randint(0, h - oh - 1)
                if np.any(self.map[y:y+oh, x:x+ow]):
                    continue
                if self._is_inside(start, x, y, None, ow, oh, None) or self._is_inside(goal, x, y, None, ow, oh, None):
                    continue
                self.map[y:y+oh, x:x+ow] = 1
                self._add_obstacle((x, y, ow, oh))
                count += ow * oh
                if self.max_count and count >= self.max_count:
                    break
                if count >= max_obs:
                    break

    def _generate_multi_narrow(self, start, goal):
        if self.is_3d:
            self._generate_random_obstacles(start, goal)
        else:
            w, h = self.map_size
            corridor_width = 2
            spacing = 6
            for i in range(spacing, h - spacing, spacing + corridor_width):
                self.map[i:i+spacing, :] = 1
            self._carve_corridor(start, goal)

    def _carve_corridor(self, start, goal):
        if not self.is_3d:
            y0, x0 = start[1], start[0]
            y1, x1 = goal[1], goal[0]
            for x in range(min(x0, x1), max(x0, x1)+1):
                self.map[y0, x] = 0
            for y in range(min(y0, y1), max(y0, y1)+1):
                self.map[y, x1] = 0

    def _generate_maze(self):
        width, height = self.map_size
        if width % 2 == 0: width += 1
        if height % 2 == 0: height += 1

        maze = np.ones((height, width), dtype=np.uint8)
        sx, sy = 1, 1
        maze[sy, sx] = 0

        walls = [(sx + dx, sy + dy) for dx, dy in [(-2, 0), (2, 0), (0, -2), (0, 2)]
                 if 0 < sx + dx < width and 0 < sy + dy < height]

        while walls:
            wx, wy = walls.pop(random.randint(0, len(walls) - 1))
            if maze[wy, wx] == 1:
                neighbors = [(wx + dx, wy + dy) for dx, dy in [(-2, 0), (2, 0), (0, -2), (0, 2)]
                             if 0 < wx + dx < width and 0 < wy + dy < height and maze[wy + dy, wx + dx] == 0]
                if len(neighbors) == 1:
                    nx, ny = neighbors[0]
                    maze[(wy + ny) // 2, (wx + nx) // 2] = 0
                    maze[wy, wx] = 0
                    for dx, dy in [(-2, 0), (2, 0), (0, -2), (0, 2)]:
                        nx, ny = wx + dx, wy + dy
                        if 0 < nx < width and 0 < ny < height and maze[ny, nx] == 1:
                            walls.append((nx, ny))

        return maze

In [58]:
class MapIO:
    @staticmethod
    def save_map(map_data: Map, filename: str) -> None:
        """Save Map object to a binary file."""
        with open(filename, 'wb') as f:
            pickle.dump(map_data, f)

    @staticmethod
    def load_map(filename: str) -> Map:
        """Load Map object from a binary file."""
        if not os.path.exists(filename):
            raise FileNotFoundError(f"Map file not found: {filename}")
        with open(filename, 'rb') as f:
            return pickle.load(f)

In [57]:
def visualize_map_shapes(
    map_array: np.ndarray,
    start: Optional[Tuple[int, ...]] = None,
    goal: Optional[Tuple[int, ...]] = None,
    obs:List[Tuple[int, ...]] = None,
    path: Optional[List[Tuple[float, ...]]] = None,
    nodes: Optional[List[Tuple[float, ...]]] = None,
    edges: Optional[List[Tuple[float, ...]]] = None,
    title: str = "Map Visualization"
):
    fig = go.Figure()

    if map_array.ndim == 2:
        height, width = map_array.shape
        
        for x, y, w, h in obs:
                fig.add_shape(
                    type="rect",
                    x0=x, x1=x+w, y0=y, y1=y+h,
                    fillcolor="purple",opacity=0.5,
                    line=dict(width=0)
                )

        # 방문 노드
        if nodes:
            vx, vy = zip(*nodes)
            fig.add_trace(go.Scatter(
                x=vx, y=vy, mode="markers",
                marker=dict(size=4, color="blue"),
                name="nodes"
            ))

        # 경로
        if path:
            px, py = zip(*path)
            fig.add_trace(go.Scatter(
                x=px, y=py, mode="lines+markers",
                line=dict(color="green"),
                marker=dict(size=6),
                name="Path"
            ))

        # 엣지 (연결 정보)
        if edges:
            for parent, child in edges:
                fig.add_trace(go.Scatter(
                    x=[parent.position[0], child.position[0]], y=[parent.position[1], child.position[1]],
                    mode="lines",
                    line=dict(color="lightblue", width=1),
                    showlegend=False,
                    hoverinfo="skip"
                ))


        # 시작/목표
        if start:
            fig.add_trace(go.Scatter(
                x=[start[0]], y=[start[1]], mode="markers",
                marker=dict(size=10, color="red"),
                name="Start"
            ))

        if goal:
            fig.add_trace(go.Scatter(
                x=[goal[0]], y=[goal[1]], mode="markers",
                marker=dict(size=10, color="orange"),
                name="Goal"
            ))

        fig.add_shape(
            type="rect",
            x0=0, y0=0,
            x1=width, y1=height,
            line=dict(color="white", width=3),
            fillcolor="rgba(0,0,0,0)",  # 투명 내부
            layer="above"
        )

        fig.update_layout(
            title=title,
            xaxis=dict(scaleanchor="y", showgrid=False),
            # yaxis=dict(showgrid=False, autorange="reversed"),
            yaxis=dict(showgrid=False),
            height=600, width=600
        )

    elif map_array.ndim == 3:
        z, y, x = map_array.nonzero()
        x, y, z = list(x), list(y), list(z)

        # for x,y,w,h in obs:
        #     fig.add_trace(go.Mesh3d(
        #         x=x, y=y, z=z,
        #         color='black',
        #         opacity=1.0,
        #         alphahull=0,
        #         name='Obstacles'
        #     ))

        fig.add_trace(go.Mesh3d(
            x=x, y=y, z=z,
            color='black',
            opacity=1.0,
            alphahull=0,
            name='Obstacles'
        ))

        if nodes:
            vx, vy, vz = zip(*nodes)
            fig.add_trace(go.Scatter3d(
                x=vx, y=vy, z=vz,
                mode='markers',
                marker=dict(size=2, color='blue'),
                name='Visited'
            ))

        # 엣지 (연결 정보)
        if edges:
            for parent, child in edges:
                fig.add_trace(go.Scatter3d(
                    x=[parent.position[0], child.position[0]],
                    y=[parent[1].position, child.position[1]],
                    z=[parent[2].position, child.position[2]],
                    mode='lines',
                    line=dict(color='lightblue', width=2),
                    showlegend=False,
                    hoverinfo="skip"
                ))

        if path:
            px_, py_, pz_ = zip(*path)
            fig.add_trace(go.Scatter3d(
                x=px_, y=py_, z=pz_,
                mode='lines+markers',
                marker=dict(size=3, color='green'),
                name='Path'
            ))

        if start:
            fig.add_trace(go.Scatter3d(
                x=[start[0]], y=[start[1]], z=[start[2]],
                mode='markers',
                marker=dict(size=5, color='red'),
                name='Start'
            ))

        if goal:
            fig.add_trace(go.Scatter3d(
                x=[goal[0]], y=[goal[1]], z=[goal[2]],
                mode='markers',
                marker=dict(size=5, color='orange'),
                name='Goal'
            ))

        fig.update_layout(
            title=title,
            scene=dict(aspectmode='data'),
            height=700, width=700
        )

        

    fig.show()


In [ ]:
gen = MapGenerator(map_type="random", map_size=(100, 100), min_obstacle_size=(5, 5), max_obstacle_size=(15, 15), obstacle_percent=0.32)
m = gen.generate(start=(1, 1), goal=(99, 99))
visualize_map_shapes(m.grid, obs=m.obstacles, start=m.start, goal=m.goal)

In [ ]:
def generate_box_with_narrow_entry(width: int, height: int, entry_side: str = "left", entry_pos: int = 10, entry_width: int = 1) -> Map:
    grid = np.zeros((height, width), dtype=np.uint8)
    obstacles = []
    start, goal = (80,50), (50,50)
    

    grid[27:30, 52:70] = 1  
    obstacles.append((27, 52, 3, 18))
    grid[27:30, 30:48] = 1  
    obstacles.append((27, 30, 3, 18))
    
    grid[70:73, 30:70] = 1
    obstacles.append((70, 30, 3, 40))

    grid[30:70, 30:33] = 1
    obstacles.append((30, 30, 40, 3))
    grid[30:70, 67:70] = 1  
    obstacles.append((30, 67, 40, 3))


    # Obstacle extraction

    return Map(
        grid=grid,
        start=start,
        goal=goal,
        obstacles=obstacles,
        size=(width, height)
    )

In [ ]:
def generate_box_with_narrow_entry(width: int, height: int, entry_side: str = "left", entry_pos: int = 10, entry_width: int = 1) -> Map:
    grid = np.zeros((height, width), dtype=np.uint8)
    obstacles = []
    start, goal = (29,40), (90,10)
    


    grid[17:20, 20:80] = 1
    obstacles.append((17, 20, 3, 60))
    grid[37:40, 0:50] = 1
    obstacles.append((37, 0, 3, 50))
    grid[57:60, 20:100] = 1
    obstacles.append((57, 20, 3, 80))
    grid[77:80, 20:70] = 1
    obstacles.append((77, 0, 3, 80))

    grid[20:37, 47:60] = 1  
    obstacles.append((20, 47, 17, 3))
    grid[30:57, 62:65] = 1 
    obstacles.append((30, 62, 27, 3))
    grid[10:45, 80:83] = 1 
    obstacles.append((10, 80, 35, 3))
    grid[10:45, 80:83] = 1 
    obstacles.append((70, 80, 20, 3))


    # Obstacle extraction

    return Map(
        grid=grid,
        start=start,
        goal=goal,
        obstacles=obstacles,
        size=(width, height)
    )

In [112]:
def generate_box_with_narrow_entry(width: int, height: int, entry_side: str = "left", entry_pos: int = 10, entry_width: int = 1) -> Map:
    grid = np.zeros((height, width), dtype=np.uint8)
    obstacles = []
    start, goal = (29,10), (90,80)
    
    grid[27:30, 20:80] = 1
    obstacles.append((27, 20, 3, 60))
    grid[47:50, 0:50] = 1
    obstacles.append((47, 0, 3, 50))
    grid[77:80, 20:100] = 1
    obstacles.append((77, 20, 3, 80))

    grid[30:47, 47:50] = 1  
    obstacles.append((30, 47, 17, 3))
    grid[50:77, 72:65] = 1 
    obstacles.append((50, 72, 27, 3))

    # Obstacle extraction

    return Map(
        grid=grid,
        start=start,
        goal=goal,
        obstacles=obstacles,
        size=(width, height)
    )

In [113]:
map_obj = generate_box_with_narrow_entry(width=100, height=100, entry_side="left", entry_pos=25, entry_width=2)


In [4]:
visualize_map_shapes(map_obj.grid, obs=map_obj.obstacles, start=map_obj.start, goal=map_obj.goal)

NameError: name 'visualize_map_shapes' is not defined

In [10]:
# MapIO.save_map(map_obj, "Maze_map_easy.pkl")

In [59]:
maze_map = MapIO.load_map("Maze_map_easy.pkl")
print("Start:", maze_map.start)
print("Obstacles:", len(maze_map.obstacles))
print(maze_map.grid.shape)

Start: (29, 10)
Obstacles: 5
(100, 100)


In [ ]:
maze_map = MapIO.load_map("Maze_map.pkl")
print("Start:", maze_map.start)
print("Obstacles:", len(maze_map.obstacles))
print(maze_map.grid.shape)

In [60]:
narrow_map = MapIO.load_map("Narrow_map.pkl")
print("Start:", narrow_map.start)
print("Obstacles:", len(narrow_map.obstacles))
print(narrow_map.grid.shape)

Start: (80, 50)
Obstacles: 5
(100, 100)


In [61]:
# 불러오기
multi_obs_map = MapIO.load_map("Multi_obs_map.pkl")
print("Start:", multi_obs_map.start)
print("Obstacles:", len(multi_obs_map.obstacles))
print(multi_obs_map.grid.shape)

Start: (1, 1)
Obstacles: 35
(100, 100)


In [9]:
visualize_map_shapes(multi_obs_map.grid, obs=multi_obs_map.obstacles, start=multi_obs_map.start, goal=multi_obs_map.goal)

In [156]:
# MultiMapbenchmarker
import pandas as pd
import numpy as np
import random
import time
from typing import Tuple, Literal, Union, Optional, List, Dict, NamedTuple, Callable, Any
import plotly.express as px
import math

from eoh.problems.optimization.classic_benchmark_path_planning.utils.architecture_utils import PlannerResult
class MultiMapBenchmarker:
    def __init__(
        self,
        maps: List[np.ndarray],
        name: str = "Algorithm",
        learning_mode = True,
        iter = 10,
        seed = 42,
    ):
        self.set_seed(seed)

        self.maps = maps
        self.name = name
        self.results_df: pd.DataFrame = pd.DataFrame()
        self.iter = iter
        self.learning_mode = learning_mode

        self.time_limit = 5.0
        self.success_limit = 0.8

    def run(self, algorithm) -> pd.DataFrame:
        results = []
        main_start_time = time.time()
        for i, map_ in enumerate(self.maps):
            print(f"[{time.strftime('%Y.%m.%d - %H:%M:%S')}] Map {i+1}")
            for j in range(self.iter):
                start_time = time.time()
                try:
                    output = algorithm(map_)
                except Exception as e:
                    output = {"path": [], "nodes": [], "n_nodes": 0}
                end_time = time.time()


                if isinstance(output, PlannerResult):
                    path = output.path
                    nodes = output.nodes
                    num_nodes = len(nodes)
                    success = self._is_path_valid(path, map_)
                    path_length = self._compute_path_length(path)
                    path_smoothness = self._compute_path_smoothness(path)
                elif isinstance(output, Dict):
                    path = output['path']
                    nodes = output['nodes']
                    num_nodes = len(nodes)
                    success = self._is_path_valid(path, map_)
                    path_length = self._compute_path_length(path)
                    path_smoothness = self._compute_path_smoothness(path)
                else:
                    return None, None
                
                print(f"Iteration {j+1}: Time taken: {end_time - start_time:.4f} seconds, Success: {success}")

                results.append({
                    "map_id": i,
                    "iter" : j,
                    "algorithm": self.name,
                    "success": success,
                    "time_taken": end_time - start_time,
                    "num_nodes": num_nodes,
                    "path_length": path_length,
                    "path_smoothness": path_smoothness,
                })

                time_taken_avg = np.mean([r["time_taken"] for r in results])
                success_avg = np.mean([r["success"] for r in results]) if len(results) > 6 else 1.0
                if time_taken_avg > self.time_limit:
                    print("Time taken Limit")
                    return None, None
                elif success_avg < self.success_limit:
                    print("Success Rate Limit")
                    return None, None

        print(f"Total time taken for all maps: {time.time() - main_start_time:.4f} seconds")

        self.results_df = pd.DataFrame(results)
        return self.results_df, self.get_avg()

    def _compute_path_length(self, path: List[Tuple[float, ...]]) -> float:
        if not path or len(path) < 2:
            return 0.0
        return sum(np.linalg.norm(np.array(path[i]) - np.array(path[i+1])) for i in range(len(path) - 1))
    
    def _is_path_valid(self, path: List[Tuple[float, ...]], map_) -> bool:
        if not path or len(path) < 2:
            return False
        
        if np.linalg.norm(np.array(path[0]) - np.array(map_.start)) > 0.5 or np.linalg.norm(np.array(path[-1]) - np.array(map_.goal)) > 0.5:
            if np.linalg.norm(np.array(path[0]) - np.array(map_.goal)) > 0.5 or np.linalg.norm(np.array(path[-1]) - np.array(map_.start)) > 0.5:
                return False
            
        is_3d = True if len(map_.size) > 2 else False
        for i in range(len(path) - 1):
            p1 = path[i]
            p2 = path[i + 1]
            if self._is_edge_in_obstacle(p1, p2, map_.obstacles, is_3d):
                return False
        return True
    
    def _is_in_obstacle(self, pos, obstacles, is_3d):
        for obs in obstacles:
            if is_3d:
                x, y, z, w, h, d = obs
                px, py, pz = pos
                if x <= px <= x + w and y <= py <= y + h and z <= pz <= z + d:
                    return True
            else:
                x, y, w, h = obs
                px, py = pos
                if x <= px <= x + w and y <= py <= y + h:
                    return True
        return False

    def _is_edge_in_obstacle(self, from_pos, to_pos, obstacles, is_3d, resolution=1.0):
        distance = math.dist(from_pos, to_pos)
        steps = max(1, int(distance / resolution))
        for i in range(steps + 1):
            interp = tuple(from_pos[d] + (to_pos[d] - from_pos[d]) * (i / steps) for d in range(len(from_pos)))
            if self._is_in_obstacle(interp, obstacles, is_3d):
                return True
        return False
    
    def _compute_path_smoothness(self, path: List[Tuple[float, ...]]) -> float:
        """
        Compute smoothness based on total bending angles (smaller is smoother).
        Returns a value where 1 = perfectly smooth (straight), 0 = very jagged.
        """
        if not path or len(path) < 3:
            return 1.0  # trivially smooth

        total_angle = 0.0
        for i in range(1, len(path) - 1):
            p0 = np.array(path[i - 1])
            p1 = np.array(path[i])
            p2 = np.array(path[i + 1])

            v1 = p0 - p1
            v2 = p2 - p1

            norm_v1 = np.linalg.norm(v1)
            norm_v2 = np.linalg.norm(v2)
            if norm_v1 == 0 or norm_v2 == 0:
                continue  # ignore invalid

            cosine = np.dot(v1, v2) / (norm_v1 * norm_v2)
            cosine = np.clip(cosine, -1.0, 1.0)  # numerical safety
            angle = np.arccos(cosine)
            total_angle += angle

        # Normalize to [0, 1]: smoothness = 1 / (1 + total_bend)
        return 1.0 / (1.0 + total_angle)

    def save_results(self, filename: str):
        if self.results_df.empty:
            raise RuntimeError("No results to save. Run benchmark first.")
        self.results_df.to_csv(filename, index=False)

    def plot_metrics(self, metric: str = "time_taken"):
        if self.results_df.empty:
            raise RuntimeError("No results to plot. Run benchmark first.")
        if metric not in self.results_df.columns:
            raise ValueError(f"Invalid metric: {metric}")

        fig = px.bar(
            self.results_df,
            x="map_id",
            y=metric,
            color="success",
            title=f"{self.name} - {metric} per map",
            labels={"map_id": "Map ID", metric: metric.replace('_', ' ').title()}
        )
        fig.show()

    def get_avg(self):
        if self.results_df.empty:
            raise RuntimeError("No results to save. Run benchmark first.")

        classic_summary = self.results_df.groupby('map_id').agg({
            'success': 'mean',           # 성공률
            'time_taken': 'mean',        # 평균 소요 시간
            'num_nodes': 'mean',      # 평균 노드 수
            'path_length': lambda x: x[x > 0].mean(),  # path=0 제외한 평균 경로 길이
            'path_smoothness': 'mean' # 경로 부드러움
        }).rename(columns={
            'success': 'success_rate',
            'time_taken': 'time_avg',
            'num_nodes': 'num_nodes_avg',
            'path_length': 'path_length_avg',
            'path_smoothness': 'smoothness_avg'
        }).reset_index()
        return classic_summary
    
    @staticmethod
    def get_improvement(reference_result:pd.DataFrame, results:pd.DataFrame) -> pd.DataFrame:
        improvement_df = pd.DataFrame()
        
        improvement_df['success_improvement'] = (results['success_rate'] - reference_result['success_rate']) * 100 # percent point
        improvement_df['time_improvement'] = (results['time_avg'] - reference_result['time_avg']) / reference_result['time_avg'] * -100
        improvement_df['length_improvement'] = (results['path_length_avg'] - reference_result['path_length_avg']) / reference_result['path_length_avg'] * -100
        improvement_df['smoothness_improvement'] = (results['smoothness_avg'] - reference_result['smoothness_avg']) / reference_result['smoothness_avg'] * 100

        improvement_df['objective_score'] = (
            5 * improvement_df['success_improvement'] +
            0.3 * improvement_df['time_improvement'] +
            0.2 * improvement_df['length_improvement'] +
            0.005 * improvement_df['smoothness_improvement']
            )

        return improvement_df
    
    def set_seed(self, seed: int = 42):
        random.seed(seed)                    # Python random
        np.random.seed(seed)                 # NumPy
        # torch.manual_seed(seed)              # PyTorch (CPU)
        # torch.cuda.manual_seed(seed)         # PyTorch (GPU)
        # torch.cuda.manual_seed_all(seed)     # If multiple GPUs
        # torch.backends.cudnn.deterministic = True  # CUDNN 고정
        # torch.backends.cudnn.benchmark = False     # 연산 최적화 OFF

## Generate templete

In [ ]:
# --- PlannerResult structure ---
class PlannerResult(NamedTuple):
    success: bool                       # Path navigation success or not
    path: List[Tuple[float, ...]]       # Final path from start to goal
    nodes: List[Node]                   # All explored nodes
    edges: List[Tuple[Node, Node]]      # Parent-child connections


# --- Node class ---
class Node:
    def __init__(self, position, parent=None, cost=0.0):
        self.position = position        # Tuple[float, ...] → 2D: (x,y), 3D: (x,y,z)
        self.parent = parent            # Node or None
        self.cost = cost                # Path cost
        self.children = []
        self.valid = True               # For collision checking etc.

    #### Create additional methods if needed ####

# --- Main Planner ---
class Planner:
    def __init__(self, max_iter: int = 5000):
        self.max_iter = max_iter

    def plan(self, map: Map) -> PlannerResult:
        bounds = map.size                  # Tuple[int, ...]: (W,H) or (W,H,D)
        start_position = map.start         # Tuple[float, ...] (W,H) or (W,H,D)
        goal_position = map.goal           # Tuple[float, ...] (W,H) or (W,H,D)
        obstacles = map.obstacles          # Rectangular blocks: 2D=(x,y,w,h), 3D=(x,y,z,w,h,d)

        is_3d = len(bounds) == 3

        # Core data
        success_state = False # Path navigation success or not
        extracted_path: List[Tuple[float, ...]] = [] # Final path from start to goal
        nodes: List[Node] = [] # All explored nodes
        edges: List[Tuple[Node, Node]] = [] # Parent-child connections

        #### Place holder: path planning logic ####

        return PlannerResult(
            success=success_state,
            path=extracted_path,
            nodes=nodes,
            edges=edges
        )
    
    def _is_in_obstacle(self, pos, obstacles, is_3d):
        for obs in obstacles:
            if is_3d:
                x, y, z, w, h, d = obs
                px, py, pz = pos
                if x <= px <= x + w and y <= py <= y + h and z <= pz <= z + d:
                    return True
            else:
                x, y, w, h = obs
                px, py = pos
                if x <= px <= x + w and y <= py <= y + h:
                    return True
        return False

    def _is_edge_in_obstacle(self, from_pos, to_pos, obstacles, is_3d, resolution=1.0):
        distance = math.dist(from_pos, to_pos)
        steps = max(1, int(distance / resolution))
        for i in range(steps + 1):
            interp = tuple(from_pos[d] + (to_pos[d] - from_pos[d]) * (i / steps) for d in range(len(from_pos)))
            if self._is_in_obstacle(interp, obstacles, is_3d):
                return True
        return False

In [8]:
json_path = './eoh/src/eoh/problems/optimization/classic_benchmark_path_planning/utils/classic_method.json'

In [282]:
result = {
    "algorithm": "Improved-RRT*-Connect",
    "algorithm_description": "Improved RRT*-Connect is a bidirectional, asymptotically optimal planner that enhances RRT*-Connect by incorporating informed heuristic sampling, adaptive step size, node rejection, and pruning. It accelerates convergence and improves success rate in narrow, obstacle-rich environments.",
    "planning_mechanism": "The planner grows two trees from start and goal using informed sampling. During expansion, it adaptively adjusts the step size near obstacles, rejects inefficient new nodes, and prunes branches that cannot contribute to an improved solution. The planner rewires nearby nodes only if doing so reduces path cost, and updates the current best path whenever a successful connection is found.",
    "code": '''
class Node:
    def __init__(self, position, parent=None, cost=0.0):
        self.position = position
        self.parent = parent
        self.cost = cost
        self.children = []
        self.valid = True

    def add_child(self, child):
        self.children.append(child)
        child.parent = self

    def path_from_root(self):
        node, path = self, []
        while node:
            path.append(node.position)
            node = node.parent
        return path[::-1]


class Planner:
    def __init__(self, max_iter=5000, step_size=5.0, rewire_radius=15.0):
        self.max_iter = max_iter
        self.base_step = step_size
        self.rewire_radius = rewire_radius

    def plan(self, map):
        import math, random, numpy as np

        bounds = map.size
        start, goal = map.start, map.goal
        obstacles = map.obstacles
        is_3d = len(bounds) == 3
        dim = len(bounds)

        tree_a, tree_b = [Node(start)], [Node(goal)]
        nodes = [tree_a[0], tree_b[0]]
        edges = []
        success, c_best, best_path = False, float("inf"), []
        c_min = math.dist(start, goal)

        for i in range(self.max_iter):
            tree_a, tree_b = (tree_a, tree_b) if i % 2 == 0 else (tree_b, tree_a)

            sample = self._informed_sample(start, goal, c_best, c_min, bounds, dim)
            nearest = min(tree_a, key=lambda n: math.dist(n.position, sample))

            step = self._adaptive_step(nearest.position, sample, obstacles, is_3d)
            new_pos = self._steer(nearest.position, sample, step)

            if self._is_in_obstacle(new_pos, obstacles, is_3d):
                continue
            if self._is_edge_in_obstacle(nearest.position, new_pos, obstacles, is_3d):
                continue

            cost = nearest.cost + math.dist(nearest.position, new_pos)
            if cost + math.dist(new_pos, goal) >= c_best:
                continue  # pruning

            new_node = Node(new_pos, nearest, cost)
            nearest.add_child(new_node)
            tree_a.append(new_node)
            nodes.append(new_node)
            edges.append((nearest, new_node))

            near_nodes = [n for n in tree_a if math.dist(n.position, new_node.position) <= self.rewire_radius]
            for near in near_nodes:
                new_cost = new_node.cost + math.dist(new_node.position, near.position)
                if new_cost < near.cost and not self._is_edge_in_obstacle(new_node.position, near.position, obstacles, is_3d):
                    if near.parent:
                        near.parent.children.remove(near)
                        edges.remove((near.parent, near))
                    near.parent = new_node
                    near.cost = new_cost
                    new_node.add_child(near)
                    edges.append((new_node, near))

            # Try to connect to the other tree
            other_nearest = min(tree_b, key=lambda n: math.dist(n.position, new_node.position))
            connect_cost = new_node.cost + math.dist(new_node.position, other_nearest.position) + other_nearest.cost
            if connect_cost < c_best and not self._is_edge_in_obstacle(new_node.position, other_nearest.position, obstacles, is_3d):
                c_best = connect_cost
                path_a = new_node.path_from_root()
                path_b = other_nearest.path_from_root()
                best_path = path_a + path_b[::-1]
                success = True

        return PlannerResult(success=success, path=best_path, nodes=nodes, edges=edges)

    def _informed_sample(self, start, goal, c_best, c_min, bounds, dim):
        import numpy as np, math, random
        if c_best == float("inf"):
            return tuple(random.uniform(0, bounds[d]) for d in range(dim))
        x_center = [(s + g) / 2 for s, g in zip(start, goal)]
        a1 = np.array(goal) - np.array(start)
        a1 = a1 / np.linalg.norm(a1)
        M = np.outer(a1, np.eye(dim)[0])
        U, _, Vt = np.linalg.svd(M)
        C = U @ np.diag([1] * (dim - 1) + [np.linalg.det(U) * np.linalg.det(Vt)]) @ Vt
        r1 = c_best / 2
        r2 = math.sqrt(c_best**2 - c_min**2) / 2
        L = np.diag([r1] + [r2] * (dim - 1))
        while True:
            x_ball = np.random.normal(0, 1, dim)
            x_ball /= np.linalg.norm(x_ball)
            x_ball *= random.random() ** (1 / dim)
            x_rand = C @ L @ x_ball + x_center
            if all(0 <= x_rand[d] <= bounds[d] for d in range(dim)):
                return tuple(x_rand)

    def _adaptive_step(self, from_pos, to_pos, obstacles, is_3d):
        import math
        distance = math.dist(from_pos, to_pos)
        steps = max(2, int(distance))
        for i in range(1, steps + 1):
            interp = tuple(from_pos[d] + (to_pos[d] - from_pos[d]) * (i / steps) for d in range(len(from_pos)))
            if self._is_in_obstacle(interp, obstacles, is_3d):
                return max(self.base_step * 0.3, 1.0)
        return self.base_step

    def _steer(self, from_pos, to_pos, step):
        import math
        dist = math.dist(from_pos, to_pos)
        if dist <= step:
            return to_pos
        return tuple(from_pos[d] + (to_pos[d] - from_pos[d]) * step / dist for d in range(len(from_pos)))

    def _is_in_obstacle(self, pos, obstacles, is_3d):
        for obs in obstacles:
            if is_3d:
                x, y, z, w, h, d = obs
                px, py, pz = pos
                if x <= px <= x + w and y <= py <= y + h and z <= pz <= z + d:
                    return True
            else:
                x, y, w, h = obs
                px, py = pos
                if x <= px <= x + w and y <= py <= y + h:
                    return True
        return False

    def _is_edge_in_obstacle(self, from_pos, to_pos, obstacles, is_3d, resolution=1.0):
        import math
        distance = math.dist(from_pos, to_pos)
        steps = max(1, int(distance / resolution))
        for i in range(steps + 1):
            interp = tuple(from_pos[d] + (to_pos[d] - from_pos[d]) * (i / steps) for d in range(len(from_pos)))
            if self._is_in_obstacle(interp, obstacles, is_3d):
                return True
        return False

    '''
}

# 저장
with open(json_path, "a") as f:
    json.dump(result, f, indent=4)
    f.write(",\n")

In [63]:
json_path = './eoh/src/eoh/problems/optimization/classic_benchmark_path_planning/utils/classic_method.json'
with open(json_path, "r") as f:
    classic_method = json.load(f)

print(classic_method[0].keys())

dict_keys(['algorithm', 'algorithm_description', 'planning_mechanism', 'code'])


In [ ]:
Instruction : Generate Improved-RRT*-Connect algorithm

Refer to below architecture:
!!!!!generate Only class Node and Planner!!!!!!
# --- Node class ---
class Node:
    def __init__(self, position, parent=None, cost=0.0):
        self.position = position        # Tuple[float, ...] → 2D: (x,y), 3D: (x,y,z)
        self.parent = parent            # Node or None
        self.cost = cost                # Path cost
        self.children = []
        self.valid = True               # For collision checking etc.

    #### Create additional methods if needed ####

# --- PlannerResult structure ---
class PlannerResult(NamedTuple):
    success: bool                       # Path navigation success or not
    path: List[Tuple[float, ...]]       # Final path from start to goal
    nodes: List[Node]                   # All explored nodes
    edges: List[Tuple[Node, Node]]      # Parent-child connections

# --- Main Planner ---
class Planner:
    def __init__(self, max_iter: int = 5000, step_size: float=5.0):
        self.max_iter = max_iter
        self.step_size = step_size

    def plan(self, map: Map) -> PlannerResult:
        bounds = map.size                  # Tuple[int, ...]: (W,H) or (W,H,D)
        start_position = map.start         # Tuple[float, ...] (W,H) or (W,H,D)
        goal_position = map.goal           # Tuple[float, ...] (W,H) or (W,H,D)
        obstacles = map.obstacles          # Rectangular blocks: 2D=(x,y,w,h), 3D=(x,y,z,w,h,d)

        is_3d = len(bounds) == 3

        # Core data
        success_state = False # Path navigation success or not
        extracted_path: List[Tuple[float, ...]] = [] # Final path from start to goal
        nodes: List[Node] = [] # All explored nodes
        edges: List[Tuple[Node, Node]] = [] # Parent-child connections

        #### Place holder: path planning logic ####

        return PlannerResult(
            success=success_state,
            path=extracted_path,
            nodes=nodes,
            edges=edges
        )

    def _is_in_obstacle(self, pos, obstacles, is_3d):
        for obs in obstacles:
            if is_3d:
                x, y, z, w, h, d = obs
                px, py, pz = pos
                if x <= px <= x + w and y <= py <= y + h and z <= pz <= z + d:
                    return True
            else:
                x, y, w, h = obs
                px, py = pos
                if x <= px <= x + w and y <= py <= y + h:
                    return True
        return False

    def _is_edge_in_obstacle(self, from_pos, to_pos, obstacles, is_3d, resolution=1.0):
        distance = math.dist(from_pos, to_pos)
        steps = max(1, int(distance / resolution))
        for i in range(steps + 1):
            interp = tuple(from_pos[d] + (to_pos[d] - from_pos[d]) * (i / steps) for d in range(len(from_pos)))
            if self._is_in_obstacle(interp, obstacles, is_3d):
                return True
        return False

### Objective:
- Improve path planning performance in terms of:
  - Planning efficiency
  - Path quality
  - Robustness
  - Success rate
  - Path smoothness
  - Path lengths
  - Reduce search time

### Constraints:
- Implement it in Python.
- You DO NOT NEED to declare the any imports.
- When connecting nodes and adding edges in the planner, always perform two critical checks:
1.Collision check for the node position: Ensure that the new node itself does not lie inside any obstacle.
2.Edge-obstacle intersection check: Before adding an edge between two nodes, verify that the straight-line path between them does not intersect or pass through any obstacle.
- DO NOT OVER MAP BOUND
- After code generation, you must review the code to ensure it is syntactically correct, logically coherent, and executable within the expected environment.
- At the top of your response, write an description of the algorithm in curly braces {}, followed by a concise explanation of the planning mechanism in angle brackets <>.
- Both the description and the planning mechanism should be placed outside and above the code block.
- Output the code block containing the implementation only.
⚠️ Do not give additional explanations.


In [32]:
filtered_sorted_algorithms[0].keys()

dict_keys(['operator', 'algorithm_description', 'planning_mechanism', 'code', 'objective', 'time_improvement', 'length_improvement', 'other_inf', 'success_rate'])

In [21]:
for alg in classic_method:
    if alg['algorithm'] == 'BI-RRT*':
        print("alg load")
        result = alg

alg load


In [ ]:
for alg in filtered_sorted_algorithms:
    if alg['objective'] == -4.59249:
        print("alg load")
        result = alg

code_string = result['code']
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    planning_module = types.ModuleType("planning_module")
    exec(import_string+code_string, planning_module.__dict__)
    sys.modules[planning_module.__name__] = planning_module
    p = planning_module.Planner(max_iter=5000, step_size=5.0)


In [23]:
p = Planner(max_iter=5000, step_size=5)
# p = planning_module.Planner()

NameError: name 'Planner' is not defined

In [24]:
result = p.plan(map=multi_obs_map)

In [25]:
result2 = p.plan(map=maze_map)

In [26]:
result3 = p.plan(map=narrow_map)

In [27]:
visualize_map_shapes(multi_obs_map.grid, obs=multi_obs_map.obstacles, start=multi_obs_map.start, goal=multi_obs_map.goal,
                      path=result.path, nodes=list(map(lambda x: x.position, result.nodes)), edges=result.edges)

In [28]:
visualize_map_shapes(maze_map.grid, obs=maze_map.obstacles, start=maze_map.start, goal=maze_map.goal, path=result2.path, nodes=list(map(lambda x: x.position, result2.nodes)), edges=result2.edges)

In [29]:
visualize_map_shapes(narrow_map.grid, obs=narrow_map.obstacles, start=narrow_map.start, goal=narrow_map.goal, path=result3.path, nodes=list(map(lambda x: x.position, result3.nodes)), edges=result3.edges)

In [306]:
bench2 = MultiMapBenchmarker(
    maps=[multi_obs_map, maze_map, narrow_map],
    name="MyRRT"
)
_, avg_res = bench2.run(p.plan)

Map 1, Iteration 1: Time taken: 4.5771 seconds
Map 1, Iteration 2: Time taken: 4.7441 seconds
Map 1, Iteration 3: Time taken: 4.6237 seconds
Map 1, Iteration 4: Time taken: 4.5852 seconds
Map 1, Iteration 5: Time taken: 3.8997 seconds
Map 1, Iteration 6: Time taken: 4.4552 seconds
Map 1, Iteration 7: Time taken: 4.7395 seconds
Map 1, Iteration 8: Time taken: 4.6186 seconds
Map 1, Iteration 9: Time taken: 4.5787 seconds
Map 1, Iteration 10: Time taken: 4.4073 seconds
Map 2, Iteration 1: Time taken: 4.9983 seconds
Map 2, Iteration 2: Time taken: 5.8869 seconds
Map 2, Iteration 3: Time taken: 6.7470 seconds
Map 2, Iteration 4: Time taken: 6.2999 seconds
Map 2, Iteration 5: Time taken: 5.4660 seconds
Map 2, Iteration 6: Time taken: 6.6855 seconds
Map 2, Iteration 7: Time taken: 6.0104 seconds
Map 2, Iteration 8: Time taken: 5.8618 seconds
Map 2, Iteration 9: Time taken: 5.9092 seconds
Map 2, Iteration 10: Time taken: 7.0804 seconds
Map 3, Iteration 1: Time taken: 8.1677 seconds
Map 3, Iter

In [ ]:
def my_dummy_planner(map_array, start, goal):
    return {
        "path": [start, goal],
        "visited": [start],
        "nodes": 2
    }

bench = MultiMapBenchmarker(
    maps=[multi_obs_map, maze_map, narrow_map],
    algorithm=p.plan,
    name="MyRRT"
)


df = bench.run()
print(df)

In [67]:
import_string ='''
from typing import Tuple, Literal, Union, Optional, List, Dict, NamedTuple, Callable, Any, Set, TYPE_CHECKING, Type
import time
from queue import Queue
import numpy as np
import random
import math
import sys
import os
from eoh.problems.optimization.classic_benchmark_path_planning.utils.architecture_utils import PlannerResult, Map

'''

In [64]:
def get_exp_result(path, ref_avg):
    with open(path, "r") as f:
        data = json.load(f)
        filtered_sorted_algorithms = sorted(
        [alg for alg in data if alg.get('operator') != 'initial'],
        key=lambda x: x.get('objective', float('inf'))
        )

    len(filtered_sorted_algorithms)

    # ref_avg 앞에서 선언됨
    maps = [multi_obs_map, maze_map, narrow_map]
    benchmarker = MultiMapBenchmarker(maps=maps, iter=100)

    g_total_df = pd.DataFrame()

    for method in filtered_sorted_algorithms:
        code_string = method['code']
        namedf = pd.DataFrame()
        namedf['alg_name'] = [method['objective']]* len(maps)
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            planning_module = types.ModuleType("planning_module")
            exec(import_string+code_string, planning_module.__dict__)
            sys.modules[planning_module.__name__] = planning_module
            try:
                planner = planning_module.Planner(max_iter=5000)
            except:
                continue
            res, avg_rest = benchmarker.run(planner.plan)

            if avg_rest is None: continue
            imp_res = MultiMapBenchmarker.get_improvement(ref_avg, avg_rest)

            res_df = pd.concat([namedf, avg_rest, imp_res], axis=1)
            g_total_df = pd.concat([g_total_df, res_df], axis=0)

    return g_total_df

In [65]:
json_path = './eoh/src/eoh/problems/optimization/classic_benchmark_path_planning/utils/classic_method.json'
with open(json_path, "r") as f:
    classic_method = json.load(f)

print(classic_method[0].keys())

dict_keys(['algorithm', 'algorithm_description', 'planning_mechanism', 'code'])


In [15]:
classic_method[0]['algorithm']

'RRT'

In [13]:
for method in classic_method:
    if method['algorithm'] == 'RRT':
        print(method['code'])


class Node:
    def __init__(self, position, parent=None, cost=0.0):
        self.position = position
        self.parent = parent
        self.cost = cost
        self.children = []
        self.valid = True

    def add_child(self, child_node):
        self.children.append(child_node)


class Planner:
    def __init__(self, max_iter: int = 5000, step_size: float = 3.0, goal_sample_rate: float = 0.05):
        self.max_iter = max_iter
        self.step_size = step_size
        self.goal_sample_rate = goal_sample_rate

    def plan(self, map) -> PlannerResult:
        import random
        import math

        bounds = map.size
        start_position = map.start
        goal_position = map.goal
        obstacles = map.obstacles
        is_3d = len(bounds) == 3

        success_state = False
        extracted_path = []
        nodes = []
        edges = []

        root = Node(start_position)
        nodes.append(root)

        for _ in range(self.max_iter):
            # Goal biasing


In [68]:
maps = [multi_obs_map, maze_map, narrow_map]
benchmarker = MultiMapBenchmarker(maps=maps, iter=100)

total_df = pd.DataFrame()

for method in classic_method[:-2]:
    code_string = method['code']
    namedf = pd.DataFrame()
    namedf['alg_name'] = [method['algorithm']]* len(maps)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        planning_module = types.ModuleType("planning_module")
        exec(import_string+code_string, planning_module.__dict__)
        sys.modules[planning_module.__name__] = planning_module
        planner = planning_module.Planner(max_iter=5000)
        res, avg_rest = benchmarker.run(planner.plan)
        if method['algorithm'] == 'RRT':
            ref_avg = avg_rest
            
        if avg_rest is None: continue
        imp_res = MultiMapBenchmarker.get_improvement(ref_avg, avg_rest)

        res_df = pd.concat([namedf, avg_rest, imp_res], axis=1)
        total_df = pd.concat([total_df, res_df], axis=0)

total_df


[2025.08.20 - 14:48:29] Map 1
Iteration 1: Time taken: 0.0271 seconds, Success: True
Iteration 2: Time taken: 0.0220 seconds, Success: True
Iteration 3: Time taken: 0.0205 seconds, Success: True
Iteration 4: Time taken: 0.0190 seconds, Success: True
Iteration 5: Time taken: 0.0120 seconds, Success: True
Iteration 6: Time taken: 0.0100 seconds, Success: True
Iteration 7: Time taken: 0.0111 seconds, Success: True
Iteration 8: Time taken: 0.0516 seconds, Success: True
Iteration 9: Time taken: 0.0260 seconds, Success: True
Iteration 10: Time taken: 0.0573 seconds, Success: True
Iteration 11: Time taken: 0.0250 seconds, Success: True
Iteration 12: Time taken: 0.0241 seconds, Success: True
Iteration 13: Time taken: 0.0231 seconds, Success: True
Iteration 14: Time taken: 0.0486 seconds, Success: True
Iteration 15: Time taken: 0.0290 seconds, Success: True
Iteration 16: Time taken: 0.0160 seconds, Success: True
Iteration 17: Time taken: 0.0145 seconds, Success: True
Iteration 18: Time taken: 0

,alg_name,map_id,success_rate,time_avg,num_nodes_avg,path_length_avg,smoothness_avg,success_improvement,time_improvement,length_improvement,smoothness_improvement,objective_score
0,RRT,0,1.00,0.034351,461.98,188.080816,0.006254,0.0,-0.000000,-0.000000,0.000000,0.000000
1,RRT,1,1.00,0.207098,1637.59,303.968509,0.003852,0.0,-0.000000,-0.000000,0.000000,0.000000
2,RRT,2,1.00,0.071277,859.41,150.713378,0.007887,0.0,-0.000000,-0.000000,0.000000,0.000000
0,RRT*,0,1.00,0.072504,447.98,157.771191,0.014527,0.0,-111.067146,16.115213,132.299872,-29.435602
1,RRT*,1,1.00,0.421743,1630.00,225.665909,0.011059,0.0,-103.643854,25.760103,187.116241,-25.005554
2,RRT*,2,1.00,0.127964,796.67,116.530178,0.021042,0.0,-79.529022,22.680933,166.805158,-18.488494
0,RRT-Connect,0,1.00,0.009228,106.12,195.985942,0.011055,0.0,73.135402,-4.203047,76.774527,21.483884
1,RRT-Connect,1,1.00,0.015609,236.56,302.131675,0.007242,0.0,92.462800,0.604284,88.002431,28.299709
2,RRT-Connect,2,1.00,0.019709,251.01,160.833474,0.014351,0.0,72.349354,-6.714797,81.963031,20.771662
0,RRT*-Connect,0,1.00,0.012265,129.62,171.903015,0.016822,0.0,64.294256,8.601516,168.993066,21.853545


In [15]:
# classic method result
grouped_avg = total_df.groupby('alg_name').mean()
a = pd.DataFrame(grouped_avg)
a

,map_id,success_rate,time_avg,num_nodes_avg,path_length_avg,smoothness_avg,success_improvement,time_improvement,length_improvement,smoothness_improvement,objective_score
alg_name,,,,,,,,,,,
BI-RRT,1.0,0.876667,0.006234,137.636667,200.871593,0.011698,-12.333333,92.565445,6.919423,90.320613,-32.061546
BI-RRT*,1.0,0.906667,0.011762,139.270000,170.273572,0.029728,-9.333333,84.689980,20.070882,396.447304,-15.263260
RRT,1.0,1.000000,0.105955,986.326667,214.254234,0.005997,0.000000,0.000000,0.000000,0.000000,0.000000
RRT*,1.0,1.000000,0.212563,958.216667,166.655759,0.015543,0.000000,-100.687319,21.518749,162.073757,-25.092077
RRT*-Connect,1.0,1.000000,0.020700,227.040000,187.996802,0.016492,0.000000,71.410167,11.617421,176.692995,24.629999
RRT-Connect,1.0,1.000000,0.014870,197.896667,219.650364,0.010883,0.000000,79.429116,-3.437853,82.246663,23.552397


In [ ]:
# expert eoh1
pop_path = "./path_planning/mobj/results/pops/population_generation_9.json"

with open(pop_path, "r") as f:
    data = json.load(f)
    filtered_sorted_algorithms = sorted(
    [alg for alg in data if alg.get('operator') != 'initial'],
    key=lambda x: x.get('objective', float('inf'))
    )

In [54]:
len(filtered_sorted_algorithms)

19

In [18]:
pop_path = "./path_planning/mobj/results/pops/population_generation_9.json"

# g_total_df
res1 = get_exp_result(pop_path, ref_avg)

[2025.08.14 - 17:10:17] Map 1
Iteration 1: Time taken: 0.0067 seconds, Success: True
Iteration 2: Time taken: 0.0080 seconds, Success: True
Iteration 3: Time taken: 0.0030 seconds, Success: True
Iteration 4: Time taken: 0.0020 seconds, Success: True
Iteration 5: Time taken: 0.0030 seconds, Success: True
Iteration 6: Time taken: 0.0055 seconds, Success: True
Iteration 7: Time taken: 0.0030 seconds, Success: True
Iteration 8: Time taken: 0.0065 seconds, Success: True
Iteration 9: Time taken: 0.0031 seconds, Success: True
Iteration 10: Time taken: 0.0030 seconds, Success: True
Iteration 11: Time taken: 0.0040 seconds, Success: True
Iteration 12: Time taken: 0.0020 seconds, Success: True
Iteration 13: Time taken: 0.0060 seconds, Success: True
Iteration 14: Time taken: 0.0045 seconds, Success: True
Iteration 15: Time taken: 0.0040 seconds, Success: True
Iteration 16: Time taken: 0.0150 seconds, Success: True
Iteration 17: Time taken: 0.0030 seconds, Success: True
Iteration 18: Time taken: 0

In [22]:
whole_df = pd.concat([total_df, g_total_df], axis=0)

In [19]:
grouped_avg = res1.groupby('alg_name').mean()
res1_ = pd.DataFrame(grouped_avg)
res1_

,map_id,success_rate,time_avg,num_nodes_avg,path_length_avg,smoothness_avg,success_improvement,time_improvement,length_improvement,smoothness_improvement,objective_score
alg_name,,,,,,,,,,,
-15.03871,1.0,1.000000,0.013579,190.490000,206.028841,0.010916,0.000000,84.496450,4.074372,80.563377,26.566626
-14.13591,1.0,1.000000,0.012308,176.363333,196.179998,0.014301,0.000000,85.018563,8.689853,134.946364,27.918271
-11.35589,1.0,1.000000,0.015621,153.813333,201.804209,0.012024,0.000000,81.004718,5.766003,99.064673,25.949939
-10.57241,1.0,1.000000,0.015665,142.246667,176.476203,0.067935,0.000000,80.321413,16.999151,1102.763275,33.010070
-9.55661,1.0,1.000000,0.012265,175.636667,196.556916,0.013523,0.000000,85.029609,8.401964,122.727117,27.802911
-9.23875,1.0,1.000000,0.015159,201.483333,206.106568,0.010811,0.000000,82.357502,3.765687,79.519746,25.857987
-8.93552,1.0,1.000000,0.014492,191.120000,197.726179,0.013339,0.000000,83.132202,7.824803,119.437740,27.101810
-7.55322,1.0,1.000000,0.015216,143.576667,178.926617,0.060363,0.000000,81.798923,15.951513,977.056609,32.615262
-6.95653,1.0,1.000000,0.017026,146.270000,176.451624,0.069735,0.000000,78.801311,16.974277,1141.531672,32.742907


In [41]:
grouped_avg = res1.groupby('alg_name').mean()
res1_ = pd.DataFrame(grouped_avg)
res1_

,map_id,success_rate,time_avg,num_nodes_avg,path_length_avg,smoothness_avg,success_improvement,time_improvement,length_improvement,smoothness_improvement,objective_score
alg_name,,,,,,,,,,,
-15.03871,1.0,1.0,0.009326,159.100000,204.043208,0.011024,0.0,85.274206,3.157838,81.585185,26.621755
-14.13591,1.0,1.0,0.013584,187.833333,199.432910,0.014364,0.0,79.222202,6.635651,130.612980,25.746856
-11.35589,1.0,1.0,0.017710,168.566667,200.654919,0.012071,0.0,72.871038,4.628734,98.803195,23.281074
-10.57241,1.0,1.0,0.015676,144.500000,183.417372,0.073720,0.0,71.111487,11.793905,1210.741930,29.745937
-9.55661,1.0,1.0,0.014326,189.333333,194.467740,0.013123,0.0,77.628343,7.268733,116.690188,25.325700
-9.23875,1.0,1.0,0.017647,194.733333,201.719375,0.010940,0.0,71.768339,3.690223,80.987813,22.673485
-8.93552,1.0,1.0,0.020223,216.100000,196.615210,0.013057,0.0,64.449679,6.557742,114.640492,21.219654
-7.55322,1.0,1.0,0.014513,152.300000,180.906028,0.059150,0.0,78.107540,14.124970,895.637587,30.735444
-6.95653,1.0,1.0,0.018969,146.633333,176.208767,0.064990,0.0,69.240656,16.097533,1099.625526,29.489831


In [20]:
pop_path = "./path_planning/mobj1/results/pops/population_generation_10.json"
res2 = get_exp_result(pop_path, ref_avg)

[2025.08.14 - 17:11:48] Map 1
Iteration 1: Time taken: 0.0140 seconds, Success: True
Iteration 2: Time taken: 0.0055 seconds, Success: True
Iteration 3: Time taken: 0.0085 seconds, Success: True
Iteration 4: Time taken: 0.0068 seconds, Success: True
Iteration 5: Time taken: 0.0070 seconds, Success: True
Iteration 6: Time taken: 0.0068 seconds, Success: True
Iteration 7: Time taken: 0.0075 seconds, Success: True
Iteration 8: Time taken: 0.0105 seconds, Success: True
Iteration 9: Time taken: 0.0040 seconds, Success: True
Iteration 10: Time taken: 0.0071 seconds, Success: True
Iteration 11: Time taken: 0.0055 seconds, Success: True
Iteration 12: Time taken: 0.0050 seconds, Success: True
Iteration 13: Time taken: 0.0050 seconds, Success: True
Iteration 14: Time taken: 0.0046 seconds, Success: True
Iteration 15: Time taken: 0.0080 seconds, Success: True
Iteration 16: Time taken: 0.0070 seconds, Success: True
Iteration 17: Time taken: 0.0055 seconds, Success: True
Iteration 18: Time taken: 0

In [21]:
grouped_avg = res2.groupby('alg_name').mean()
res2_ = pd.DataFrame(grouped_avg)
res2_

,map_id,success_rate,time_avg,num_nodes_avg,path_length_avg,smoothness_avg,success_improvement,time_improvement,length_improvement,smoothness_improvement,objective_score
alg_name,,,,,,,,,,,
-11.12807,1.0,1.0,0.014165,150.463333,178.275987,0.065757,0.0,82.567512,16.003056,1039.730273,33.169516
-7.23208,1.0,1.0,0.015964,150.246667,175.065409,0.068300,0.0,79.766113,17.512948,1097.411463,32.919481
-7.12118,1.0,1.0,0.016171,149.833333,179.089798,0.064162,0.0,80.001202,15.657992,996.836520,32.116142
-6.46310,1.0,1.0,0.016827,157.380000,174.771384,0.092487,0.0,78.564985,17.849513,1597.327534,35.126036
-6.37314,1.0,1.0,0.019131,151.493333,174.228680,0.090764,0.0,75.240021,17.964819,1580.438571,34.067163
-5.84788,1.0,1.0,0.017053,152.026667,175.898703,0.068867,0.0,78.517465,17.015470,1127.780054,32.597234
-5.82745,1.0,1.0,0.017637,149.800000,174.655085,0.087632,0.0,77.944782,17.756065,1493.730385,34.403300
-5.68873,1.0,1.0,0.015999,150.620000,175.506817,0.071090,0.0,79.906859,17.314961,1147.640995,33.173255
-4.59249,1.0,1.0,0.017230,151.350000,179.261070,0.068966,0.0,78.096334,15.601277,1121.032218,32.154316


In [44]:
grouped_avg = res2.groupby('alg_name').mean()
res2_ = pd.DataFrame(grouped_avg)
res2_

,map_id,success_rate,time_avg,num_nodes_avg,path_length_avg,smoothness_avg,success_improvement,time_improvement,length_improvement,smoothness_improvement,objective_score
alg_name,,,,,,,,,,,
-11.12807,1.0,1.0,0.018101,169.066667,178.872845,0.071576,0.0,68.718863,14.408037,1114.531229,29.069923
-7.23208,1.0,1.0,0.018395,156.566667,176.656912,0.059395,0.0,66.615815,15.162321,941.424626,27.724332
-7.12118,1.0,1.0,0.013454,138.500000,179.551406,0.067572,0.0,76.505372,13.636188,1083.145978,31.094579
-6.46310,1.0,1.0,0.015685,145.800000,176.450449,0.094685,0.0,73.459261,15.698524,1624.086204,33.297914
-6.37314,1.0,1.0,0.019289,150.966667,171.525618,0.090478,0.0,67.553778,17.883870,1543.104787,31.558431
-5.84788,1.0,1.0,0.015739,148.466667,176.679617,0.062509,0.0,72.988309,15.619567,1023.166422,30.136238
-5.82745,1.0,1.0,0.021544,165.466667,174.330491,0.080653,0.0,64.126371,16.514066,1352.295411,29.302202
-5.68873,1.0,1.0,0.012612,133.033333,177.038015,0.062646,0.0,78.540216,14.884526,949.464052,31.286290
-4.59249,1.0,1.0,0.010613,139.466667,176.483368,0.070947,0.0,82.279458,15.538505,1132.027218,33.451675


In [22]:
pop_path = "./path_planning/exp_result/basic_eoh_from_ma.json"
res3 = get_exp_result(pop_path, ref_avg)

[2025.08.14 - 17:13:21] Map 1
Iteration 1: Time taken: 0.0040 seconds, Success: True
Iteration 2: Time taken: 0.0095 seconds, Success: True
Iteration 3: Time taken: 0.0056 seconds, Success: True
Iteration 4: Time taken: 0.0135 seconds, Success: True
Iteration 5: Time taken: 0.0061 seconds, Success: True
Iteration 6: Time taken: 0.0071 seconds, Success: True
Iteration 7: Time taken: 0.0055 seconds, Success: True
Iteration 8: Time taken: 0.0050 seconds, Success: True
Iteration 9: Time taken: 0.0065 seconds, Success: True
Iteration 10: Time taken: 0.0090 seconds, Success: True
Iteration 11: Time taken: 0.0083 seconds, Success: True
Iteration 12: Time taken: 0.0075 seconds, Success: True
Iteration 13: Time taken: 0.0110 seconds, Success: True
Iteration 14: Time taken: 0.0080 seconds, Success: True
Iteration 15: Time taken: 0.0065 seconds, Success: True
Iteration 16: Time taken: 0.0040 seconds, Success: True
Iteration 17: Time taken: 0.0093 seconds, Success: True
Iteration 18: Time taken: 0

In [23]:
grouped_avg = res3.groupby('alg_name').mean()
res3_ = pd.DataFrame(grouped_avg)
res3_

,map_id,success_rate,time_avg,num_nodes_avg,path_length_avg,smoothness_avg,success_improvement,time_improvement,length_improvement,smoothness_improvement,objective_score
alg_name,,,,,,,,,,,
-7.79662,1.0,1.000000,0.020503,399.493333,209.646352,0.010898,0.000000,72.463900,1.659915,81.559675,22.478951
-0.27647,1.0,1.000000,0.024761,437.833333,178.014682,0.023868,0.000000,68.442843,16.254316,300.958834,25.288510
1.22143,1.0,1.000000,0.019501,235.353333,216.995383,0.010061,0.000000,74.591785,-1.786302,68.307099,22.361811
2.98769,1.0,0.823333,0.012558,189.803333,178.166650,0.045686,-17.666667,82.188942,16.629528,681.807312,-56.941709
3.05605,1.0,0.990000,0.022014,234.163333,179.580038,0.078811,-1.000000,68.954090,15.239381,1323.445965,25.351333
4.24450,1.0,1.000000,0.019450,348.706667,180.113654,0.021846,0.000000,74.016859,15.333347,265.660620,26.600030
4.67820,1.0,0.883333,0.012862,200.800000,177.011058,0.055240,-11.666667,82.441599,16.968520,866.471857,-25.874790
9.25217,1.0,0.953333,0.026744,240.376667,183.571365,0.055025,-4.666667,61.953208,13.390181,870.205424,2.281692
9.62493,1.0,0.960000,0.024648,236.370000,184.349425,0.053541,-4.000000,64.271048,12.867089,840.090085,6.055183


In [46]:
grouped_avg = res3.groupby('alg_name').mean()
res3_ = pd.DataFrame(grouped_avg)
res3_

,map_id,success_rate,time_avg,num_nodes_avg,path_length_avg,smoothness_avg,success_improvement,time_improvement,length_improvement,smoothness_improvement,objective_score
alg_name,,,,,,,,,,,
-7.79662,1.0,1.000000,0.015127,372.466667,207.911936,0.011163,0.000000,75.002654,0.720683,84.168157,23.065774
-0.27647,1.0,1.000000,0.021599,440.866667,180.431247,0.023325,0.000000,63.665225,13.618296,289.291218,23.269683
1.22143,1.0,1.000000,0.013797,197.700000,217.688592,0.009913,0.000000,78.038886,-4.174879,65.150992,22.902445
2.98769,1.0,0.966667,0.024254,354.133333,180.615696,0.027774,-3.333333,60.731698,13.884632,406.187130,6.360705
3.05605,1.0,0.966667,0.020013,227.400000,179.251937,0.075950,-3.333333,60.732115,13.963158,1257.943283,10.635316
4.24450,1.0,0.966667,0.028696,420.800000,181.922704,0.022098,-3.333333,47.435150,13.021391,265.393317,1.495123
4.67820,1.0,0.966667,0.024194,359.266667,184.011918,0.042612,-3.333333,61.238052,11.968399,639.701721,7.296937
9.25217,1.0,0.933333,0.019653,208.100000,182.225662,0.057646,-6.666667,61.967492,12.597211,943.455238,-7.506367
9.62493,1.0,0.966667,0.022645,237.633333,181.950697,0.047889,-3.333333,56.159865,13.028923,735.261847,6.463387


In [24]:
pop_path = "./path_planning/mobj_analysis/results/pops/population_generation_15.json"

res4 = get_exp_result(pop_path, ref_avg)


[2025.08.14 - 17:15:19] Map 1
Iteration 1: Time taken: 0.0070 seconds, Success: True
Iteration 2: Time taken: 0.0081 seconds, Success: True
Iteration 3: Time taken: 0.0119 seconds, Success: True
Iteration 4: Time taken: 0.0065 seconds, Success: True
Iteration 5: Time taken: 0.0127 seconds, Success: True
Iteration 6: Time taken: 0.0090 seconds, Success: True
Iteration 7: Time taken: 0.0100 seconds, Success: True
Iteration 8: Time taken: 0.0155 seconds, Success: True
Iteration 9: Time taken: 0.0136 seconds, Success: True
Iteration 10: Time taken: 0.0070 seconds, Success: True
Iteration 11: Time taken: 0.0177 seconds, Success: True
Iteration 12: Time taken: 0.0130 seconds, Success: True
Iteration 13: Time taken: 0.0098 seconds, Success: True
Iteration 14: Time taken: 0.0127 seconds, Success: True
Iteration 15: Time taken: 0.0071 seconds, Success: True
Iteration 16: Time taken: 0.0060 seconds, Success: True
Iteration 17: Time taken: 0.0095 seconds, Success: True
Iteration 18: Time taken: 0

In [25]:
grouped_avg = res4.groupby('alg_name').mean()
res4_ = pd.DataFrame(grouped_avg)
res4_

,map_id,success_rate,time_avg,num_nodes_avg,path_length_avg,smoothness_avg,success_improvement,time_improvement,length_improvement,smoothness_improvement,objective_score
alg_name,,,,,,,,,,,
-30.22802,1.0,1.000000,0.020153,178.890000,172.272975,0.082882,0.000000,74.079551,18.865989,1410.209034,33.048108
-29.71144,1.0,0.893333,0.015251,168.116667,170.134714,0.091659,-10.666667,79.236993,20.366602,1585.119424,-17.563318
-29.43364,1.0,1.000000,0.018318,182.923333,175.280368,0.087384,0.000000,75.643640,17.504815,1501.332135,33.700716
-29.37349,1.0,1.000000,0.017996,171.623333,176.192474,0.090116,0.000000,76.411219,17.010730,1555.111662,34.101070
-28.86429,1.0,1.000000,0.018474,182.360000,175.789529,0.088634,0.000000,74.245012,17.186261,1541.080638,33.416159
-28.18988,1.0,1.000000,0.018457,184.700000,174.365758,0.086093,0.000000,74.326507,17.854531,1482.653461,33.282126
-28.11571,1.0,1.000000,0.022930,222.260000,173.269905,0.086811,0.000000,73.244902,18.448827,1478.739993,33.056936
-28.07081,1.0,1.000000,0.020983,180.363333,172.220540,0.082946,0.000000,72.536395,18.917557,1415.137622,32.620118
-27.62859,1.0,1.000000,0.014215,163.660000,174.123197,0.085373,0.000000,80.937175,18.031205,1478.286307,35.278825


In [48]:
grouped_avg = res4.groupby('alg_name').mean()
res4_ = pd.DataFrame(grouped_avg)
res4_

,map_id,success_rate,time_avg,num_nodes_avg,path_length_avg,smoothness_avg,success_improvement,time_improvement,length_improvement,smoothness_improvement,objective_score
alg_name,,,,,,,,,,,
-30.22802,1.0,1.000000,0.030523,225.700000,172.798321,0.086034,0.000000,45.119856,17.276091,1453.660750,24.259479
-29.71144,1.0,0.933333,0.025761,243.500000,163.708291,0.083751,-6.666667,55.550238,21.818030,1399.593664,-5.306688
-29.43364,1.0,1.000000,0.022761,210.400000,173.423051,0.077161,0.000000,59.945028,17.290311,1276.361167,27.823376
-28.86429,1.0,1.000000,0.022670,216.700000,179.639245,0.098549,0.000000,57.040912,13.763457,1707.759398,28.403762
-28.11571,1.0,1.000000,0.026935,260.700000,175.288632,0.078747,0.000000,48.014144,16.035997,1334.820713,24.285546
-28.07081,1.0,1.000000,0.017275,171.800000,173.603598,0.088251,0.000000,69.682133,16.968418,1536.729579,31.981972
-27.52708,1.0,1.000000,0.017313,169.700000,172.817482,0.081494,0.000000,72.252405,17.676832,1381.706022,32.119618
-27.44347,1.0,1.000000,0.014882,157.066667,173.879018,0.080190,0.000000,75.716497,17.048129,1320.751469,32.728332
-27.23958,1.0,1.000000,0.021299,195.433333,175.219438,0.086927,0.000000,59.319886,16.077090,1485.686026,28.439814


In [26]:
pop_path = "./eoh/src/eoh/problems/optimization/classic_benchmark_path_planning/utils/database/time_db.json"

time_res = get_exp_result(pop_path, ref_avg)


[2025.08.14 - 17:19:17] Map 1
Iteration 1: Time taken: 0.0075 seconds, Success: True
Iteration 2: Time taken: 0.0085 seconds, Success: True
Iteration 3: Time taken: 0.0253 seconds, Success: True
Iteration 4: Time taken: 0.0165 seconds, Success: True
Iteration 5: Time taken: 0.0070 seconds, Success: True
Iteration 6: Time taken: 0.0166 seconds, Success: True
Iteration 7: Time taken: 0.0085 seconds, Success: True
Iteration 8: Time taken: 0.0076 seconds, Success: False
Iteration 9: Time taken: 0.0070 seconds, Success: True
Iteration 10: Time taken: 0.0090 seconds, Success: True
Iteration 11: Time taken: 0.0046 seconds, Success: True
Iteration 12: Time taken: 0.0081 seconds, Success: True
Iteration 13: Time taken: 0.0078 seconds, Success: True
Iteration 14: Time taken: 0.0093 seconds, Success: True
Iteration 15: Time taken: 0.0085 seconds, Success: True
Iteration 16: Time taken: 0.0088 seconds, Success: True
Iteration 17: Time taken: 0.0075 seconds, Success: True
Iteration 18: Time taken: 

In [27]:
grouped_avg = time_res.groupby('alg_name').mean()
time_res_ = pd.DataFrame(grouped_avg)
time_res_

,map_id,success_rate,time_avg,num_nodes_avg,path_length_avg,smoothness_avg,success_improvement,time_improvement,length_improvement,smoothness_improvement,objective_score
alg_name,,,,,,,,,,,
-29.71144,1.0,0.883333,0.014242,165.793333,168.797345,0.088776,-11.666667,80.970341,20.989990,1499.757743,-22.345444
-28.86429,1.0,1.000000,0.018775,186.713333,174.926724,0.090404,0.000000,74.481947,17.556936,1562.252027,33.667231
-28.11571,1.0,1.000000,0.023721,227.266667,174.368398,0.086252,0.000000,70.856964,17.920671,1480.779603,32.245121
-28.07081,1.0,1.000000,0.018433,174.700000,172.951863,0.085108,0.000000,76.394652,18.521329,1458.036729,33.912845
-25.46582,1.0,1.000000,0.024363,234.183333,176.697864,0.080807,0.000000,68.189859,16.654445,1363.946463,30.607579
-23.98338,1.0,1.000000,0.017669,179.316667,174.865005,0.086441,0.000000,76.094411,17.639594,1472.379705,33.718141
-21.99894,1.0,1.000000,0.018009,212.260000,178.931580,0.022142,0.000000,76.860440,15.935031,271.660414,27.603440
-20.70232,1.0,1.000000,0.024551,259.476667,177.676592,0.024710,0.000000,64.886881,16.392175,315.917593,24.324087
-20.33585,1.0,0.866667,0.015732,173.253333,169.873958,0.025771,-13.333333,79.435251,20.723113,321.983784,-37.081550


In [28]:
pop_path = "./eoh/src/eoh/problems/optimization/classic_benchmark_path_planning/utils/database/path_db.json"

path_res = get_exp_result(pop_path, ref_avg)

[2025.08.14 - 17:20:28] Map 1
Iteration 1: Time taken: 0.0080 seconds, Success: True
Iteration 2: Time taken: 0.0087 seconds, Success: False
Iteration 3: Time taken: 0.0237 seconds, Success: True
Iteration 4: Time taken: 0.0105 seconds, Success: True
Iteration 5: Time taken: 0.0205 seconds, Success: True
Iteration 6: Time taken: 0.0105 seconds, Success: True
Iteration 7: Time taken: 0.0132 seconds, Success: True
Iteration 8: Time taken: 0.0147 seconds, Success: True
Iteration 9: Time taken: 0.0133 seconds, Success: True
Iteration 10: Time taken: 0.0178 seconds, Success: True
Iteration 11: Time taken: 0.0085 seconds, Success: True
Iteration 12: Time taken: 0.0161 seconds, Success: True
Iteration 13: Time taken: 0.0125 seconds, Success: True
Iteration 14: Time taken: 0.0183 seconds, Success: True
Iteration 15: Time taken: 0.0254 seconds, Success: True
Iteration 16: Time taken: 0.0185 seconds, Success: True
Iteration 17: Time taken: 0.0113 seconds, Success: True
Iteration 18: Time taken: 

In [29]:
pop_path = "./eoh/src/eoh/problems/optimization/classic_benchmark_path_planning/utils/database/smoothness_db.json"

smoothness_res = get_exp_result(pop_path, ref_avg)

[2025.08.14 - 17:23:33] Map 1
Iteration 1: Time taken: 0.0091 seconds, Success: True
Iteration 2: Time taken: 0.0110 seconds, Success: True
Iteration 3: Time taken: 0.0095 seconds, Success: True
Iteration 4: Time taken: 0.0070 seconds, Success: True
Iteration 5: Time taken: 0.0105 seconds, Success: True
Iteration 6: Time taken: 0.0145 seconds, Success: True
Iteration 7: Time taken: 0.0105 seconds, Success: True
Iteration 8: Time taken: 0.0074 seconds, Success: True
Iteration 9: Time taken: 0.0074 seconds, Success: True
Iteration 10: Time taken: 0.0100 seconds, Success: True
Iteration 11: Time taken: 0.0135 seconds, Success: True
Iteration 12: Time taken: 0.0175 seconds, Success: True
Iteration 13: Time taken: 0.0125 seconds, Success: True
Iteration 14: Time taken: 0.0285 seconds, Success: True
Iteration 15: Time taken: 0.0234 seconds, Success: True
Iteration 16: Time taken: 0.0104 seconds, Success: True
Iteration 17: Time taken: 0.0102 seconds, Success: True
Iteration 18: Time taken: 0

In [31]:
total_df['cls_try'] = 0 # classic method
res1['cls_try'] = 1 # expert1
res2['cls_try'] = 2 # expert2
res3['cls_try'] = 3 # eoh1
res4['cls_try'] = 4 # expert_analysis1
time_res['cls_try'] = 5 # time_expert_db
path_res['cls_try'] = 6 # path_expert_db
smoothness_res['cls_try'] = 7 # smoothness_expert_db

code2label = {
    0: "classic",
    1: "expert1",
    2: "expert2",
    3: "eoh1",
    4: "expert_analysis1",
    5: "time_expert_db",
    6: "path_expert_db",
    7: "smoothness_expert_db",
}


In [30]:
def get_avg_result(df, clas_name, success_drop=False):
    # alg_name 기준으로 groupby하여 평균값 계산
    grouped_avg = df.groupby('alg_name').mean()
    total_statics = pd.DataFrame(grouped_avg)
    total_statics = total_statics.drop(["map_id"], axis=1)
    total_statics = total_statics.sort_values(by='objective_score', ascending=False)
    total_statics['cls_try'] = clas_name
    # total_statics = total_statics.sort_values(by='time_improvement', ascending=False)

    if success_drop:
        total_statics = total_statics[total_statics['success_rate'] >= 1]
    
    return total_statics

In [32]:
result_list = [total_df, res1, res2, res3, res4, time_res, path_res, smoothness_res]

In [33]:
whole_res_list = list()
for i, res in enumerate(result_list):
    whole_res_list.append(get_avg_result(res, code2label[i], True if not i==0 else False))

In [43]:
whole_res_list[3].round(3)

,success_rate,time_avg,num_nodes_avg,path_length_avg,smoothness_avg,success_improvement,time_improvement,length_improvement,smoothness_improvement,objective_score,cls_try
alg_name,,,,,,,,,,,
4.24450,1.0,0.019,348.707,180.114,0.022,0.0,74.017,15.333,265.661,26.600,eoh1
-0.27647,1.0,0.025,437.833,178.015,0.024,0.0,68.443,16.254,300.959,25.289,eoh1
16.34412,1.0,0.025,398.733,179.743,0.022,0.0,67.098,15.697,273.110,24.634,eoh1
-7.79662,1.0,0.021,399.493,209.646,0.011,0.0,72.464,1.660,81.560,22.479,eoh1
1.22143,1.0,0.020,235.353,216.995,0.010,0.0,74.592,-1.786,68.307,22.362,eoh1


In [49]:
whole_df[whole_df['cls_try'].isin(['expert_analysis1'])].sort_values(by='objective_score', ascending=False)

,success_rate,time_avg,num_nodes_avg,path_length_avg,smoothness_avg,success_improvement,time_improvement,length_improvement,smoothness_improvement,objective_score,cls_try
alg_name,,,,,,,,,,,
-27.62859,1.0,0.014215,163.660000,174.123197,0.085373,0.0,80.937175,18.031205,1478.286307,35.278825,expert_analysis1
-29.37349,1.0,0.017996,171.623333,176.192474,0.090116,0.0,76.411219,17.010730,1555.111662,34.101070,expert_analysis1
-26.19822,1.0,0.016605,176.003333,175.823083,0.079032,0.0,79.251817,17.114859,1348.738635,33.942210,expert_analysis1
-26.11907,1.0,0.018841,175.173333,174.654474,0.091260,0.0,74.825897,17.786334,1572.078399,33.865428,expert_analysis1
-26.35191,1.0,0.018436,177.183333,174.368569,0.088564,0.0,75.043718,17.857679,1533.822874,33.753766,expert_analysis1
-29.43364,1.0,0.018318,182.923333,175.280368,0.087384,0.0,75.643640,17.504815,1501.332135,33.700716,expert_analysis1
-28.86429,1.0,0.018474,182.360000,175.789529,0.088634,0.0,74.245012,17.186261,1541.080638,33.416159,expert_analysis1
-28.18988,1.0,0.018457,184.700000,174.365758,0.086093,0.0,74.326507,17.854531,1482.653461,33.282126,expert_analysis1
-26.03083,1.0,0.019224,178.210000,176.680077,0.090218,0.0,74.296587,16.694638,1520.112884,33.228468,expert_analysis1


In [35]:
whole_df = pd.concat(whole_res_list, axis=0)
whole_df = whole_df.sort_values(by='objective_score', ascending=False)
# whole_df = whole_df.sort_values(by='time_improvement', ascending=False)
whole_df[:20]

,success_rate,time_avg,num_nodes_avg,path_length_avg,smoothness_avg,success_improvement,time_improvement,length_improvement,smoothness_improvement,objective_score,cls_try
alg_name,,,,,,,,,,,
-6.3318,1.0,0.016704,192.503333,173.718236,0.106156,0.0,78.331804,18.198344,1852.771326,36.403067,expert1
-27.62859,1.0,0.014215,163.660000,174.123197,0.085373,0.0,80.937175,18.031205,1478.286307,35.278825,expert_analysis1
-6.4631,1.0,0.016827,157.380000,174.771384,0.092487,0.0,78.564985,17.849513,1597.327534,35.126036,expert2
-2.05071,1.0,0.016755,150.816667,173.064249,0.087551,0.0,78.351643,18.422251,1485.599319,34.617940,expert2
-3.8343,1.0,0.018964,151.863333,173.173781,0.091802,0.0,75.857578,18.465603,1592.628594,34.413537,expert2
-5.82745,1.0,0.017637,149.800000,174.655085,0.087632,0.0,77.944782,17.756065,1493.730385,34.403300,expert2
-4.01111,1.0,0.017994,150.913333,173.911071,0.088415,0.0,77.120646,18.021195,1498.248651,34.231676,expert2
-2.91188,1.0,0.018626,152.253333,174.053331,0.086098,0.0,77.399936,18.065299,1473.616761,34.201125,expert2
-29.37349,1.0,0.017996,171.623333,176.192474,0.090116,0.0,76.411219,17.010730,1555.111662,34.101070,expert_analysis1


In [36]:
whole_df = whole_df.sort_values(by='time_improvement', ascending=False)
whole_df[:20]

,success_rate,time_avg,num_nodes_avg,path_length_avg,smoothness_avg,success_improvement,time_improvement,length_improvement,smoothness_improvement,objective_score,cls_try
alg_name,,,,,,,,,,,
BI-RRT,0.876667,0.006234,137.636667,200.871593,0.011698,-12.333333,92.565445,6.919423,90.320613,-32.061546,classic
-9.55661,1.000000,0.012265,175.636667,196.556916,0.013523,0.000000,85.029609,8.401964,122.727117,27.802911,expert1
-14.13591,1.000000,0.012308,176.363333,196.179998,0.014301,0.000000,85.018563,8.689853,134.946364,27.918271,expert1
BI-RRT*,0.906667,0.011762,139.270000,170.273572,0.029728,-9.333333,84.689980,20.070882,396.447304,-15.263260,classic
-15.03871,1.000000,0.013579,190.490000,206.028841,0.010916,0.000000,84.496450,4.074372,80.563377,26.566626,expert1
-8.93552,1.000000,0.014492,191.120000,197.726179,0.013339,0.000000,83.132202,7.824803,119.437740,27.101810,expert1
-11.12807,1.000000,0.014165,150.463333,178.275987,0.065757,0.000000,82.567512,16.003056,1039.730273,33.169516,expert2
-6.15402,1.000000,0.015010,157.233333,203.261020,0.011951,0.000000,82.370245,5.218083,97.600268,26.242692,expert1
-9.23875,1.000000,0.015159,201.483333,206.106568,0.010811,0.000000,82.357502,3.765687,79.519746,25.857987,expert1


In [34]:
col_map = {
    # "map_id": "Map ID",
    "success_rate": "성공률 (Success Rate)",
    "time_avg": "평균 시간 (Time Avg)",
    # "num_nodes_avg": "평균 노드 수 (Num Nodes Avg)",
    "path_length_avg": "평균 경로 길이 (Path Length Avg)",
    "success_improvement": "성공률 개선 (Success Improvement)",
    "time_improvement": "시간 개선 (%)",
    "length_improvement": "경로 개선 (%)",
    "objective_score": "종합 점수 (Objective Score)"
}

df_pretty = grouped_avg.rename(columns=col_map)
df_pretty = df_pretty.round(3)
df_pretty = df_pretty.sort_values("종합 점수 (Objective Score)", ascending=False)
display(df_pretty)

,map_id,성공률 (Success Rate),평균 시간 (Time Avg),num_nodes_avg,평균 경로 길이 (Path Length Avg),성공률 개선 (Success Improvement),시간 개선 (%),경로 개선 (%),종합 점수 (Objective Score)
alg_name,,,,,,,,,
BI-RRT,1.0,0.967,0.004,126.300,209.300,-3.333,88.101,6.730,26.110
RRT-Connect,1.0,1.000,0.012,175.300,213.838,0.000,69.541,4.292,21.721
BI-RRT*,1.0,0.933,0.011,142.267,172.039,-6.667,68.539,21.825,21.593
15.44831,1.0,1.000,0.013,147.633,180.308,0.000,58.762,18.807,21.390
RRT*-Connect,1.0,1.000,0.020,237.533,192.078,0.000,40.091,13.745,14.776
RRT,1.0,1.000,0.055,615.800,224.283,0.000,0.000,0.000,0.000
RRT*,1.0,1.000,0.127,688.167,174.649,0.000,-192.159,21.122,-53.423
623.91658,1.0,1.000,0.141,703.067,217.119,0.000,-252.731,3.369,-75.146
2116.54822,1.0,0.900,0.150,645.033,166.335,-10.000,-271.184,24.860,-81.383


In [ ]:
import json, os

In [94]:
def reset_obj_score(method):
    maps = [multi_obs_map, maze_map, narrow_map]
    benchmarker = MultiMapBenchmarker(maps=maps, iter=10)

    g_total_df = pd.DataFrame()

    code_string = method['code']
    namedf = pd.DataFrame()
    namedf['alg_name'] = [method['objective']]* len(maps)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        planning_module = types.ModuleType("planning_module")
        exec(import_string+code_string, planning_module.__dict__)
        sys.modules[planning_module.__name__] = planning_module
        try:
            planner = planning_module.Planner(max_iter=5000)
        except:
            pass
        res, avg_rest = benchmarker.run(planner.plan)
        
        if avg_rest is None: return 9999
        imp_res = MultiMapBenchmarker.get_improvement(ref_avg, avg_rest)

        # res_df = pd.concat([namedf, avg_rest, imp_res], axis=1)
        # g_total_df = pd.concat([g_total_df, res_df], axis=0)

        return -imp_res['objective_score'].mean()

def get_best_pop(path):
    alg_save = []

    for i in range(1,11):
        pop_path = f"{path}/population_generation_{i}.json"
        tmp_alg = []
        if os.path.exists(pop_path):
            with open(pop_path, "r") as f:
                data = json.load(f)
                tmp_alg = sorted(
                [alg for alg in data if alg.get('operator') != 'initial'],
                key=lambda x: x.get('objective', float('inf'))
                )
            new_tmp_alg = []
            for i, alg in enumerate(tmp_alg[:10]):
                tmp_alg[i]['objective'] = reset_obj_score(alg)
                new_tmp_alg.append(tmp_alg[i])
                
            new_tmp_alg = sorted(
                [alg for alg in data if alg.get('operator') != 'initial'],
                key=lambda x: x.get('objective', float('inf')))
            
            alg_save.append(new_tmp_alg[0])

    return alg_save
        

In [157]:
pop_path = "./path_planning/mobj1/results/pops"
expert1 = get_best_pop(pop_path)

[2025.08.20 - 16:08:41] Map 1
Iteration 1: Time taken: 0.0103 seconds, Success: True
Iteration 2: Time taken: 0.0195 seconds, Success: True
Iteration 3: Time taken: 0.0060 seconds, Success: True
Iteration 4: Time taken: 0.0100 seconds, Success: True
Iteration 5: Time taken: 0.0270 seconds, Success: True
Iteration 6: Time taken: 0.0080 seconds, Success: True
Iteration 7: Time taken: 0.0110 seconds, Success: True
Iteration 8: Time taken: 0.0120 seconds, Success: True
Iteration 9: Time taken: 0.0080 seconds, Success: True
Iteration 10: Time taken: 0.0090 seconds, Success: True
[2025.08.20 - 16:08:41] Map 2
Iteration 1: Time taken: 0.0170 seconds, Success: True
Iteration 2: Time taken: 0.0320 seconds, Success: True
Iteration 3: Time taken: 0.0160 seconds, Success: True
Iteration 4: Time taken: 0.0195 seconds, Success: True
Iteration 5: Time taken: 0.0210 seconds, Success: True
Iteration 6: Time taken: 0.0240 seconds, Success: True
Iteration 7: Time taken: 0.0120 seconds, Success: True
Iter

In [158]:
pop_path = "./path_planning/basic_eoh/results/pops"
basic = get_best_pop(pop_path)

[2025.08.20 - 16:09:46] Map 1
Iteration 1: Time taken: 0.0066 seconds, Success: True
Iteration 2: Time taken: 0.0050 seconds, Success: True
Iteration 3: Time taken: 0.0060 seconds, Success: True
Iteration 4: Time taken: 0.0060 seconds, Success: True
Iteration 5: Time taken: 0.0080 seconds, Success: True
Iteration 6: Time taken: 0.0050 seconds, Success: True
Iteration 7: Time taken: 0.0060 seconds, Success: False
Iteration 8: Time taken: 0.0095 seconds, Success: True
Iteration 9: Time taken: 0.0050 seconds, Success: True
Iteration 10: Time taken: 0.0120 seconds, Success: True
[2025.08.20 - 16:09:46] Map 2
Iteration 1: Time taken: 0.0330 seconds, Success: True
Iteration 2: Time taken: 0.0250 seconds, Success: True
Iteration 3: Time taken: 0.0362 seconds, Success: True
Iteration 4: Time taken: 0.0380 seconds, Success: True
Iteration 5: Time taken: 0.0280 seconds, Success: True
Iteration 6: Time taken: 0.0372 seconds, Success: True
Iteration 7: Time taken: 0.0300 seconds, Success: True
Ite

In [159]:
pop_path = "./path_planning/mobj_analysis/results/pops"
analysis = get_best_pop(pop_path)

[2025.08.20 - 16:12:44] Map 1
Iteration 1: Time taken: 0.0030 seconds, Success: True
Iteration 2: Time taken: 0.0035 seconds, Success: True
Iteration 3: Time taken: 0.0120 seconds, Success: True
Iteration 4: Time taken: 0.0060 seconds, Success: True
Iteration 5: Time taken: 0.0090 seconds, Success: True
Iteration 6: Time taken: 0.0040 seconds, Success: True
Iteration 7: Time taken: 0.0050 seconds, Success: True
Iteration 8: Time taken: 0.0100 seconds, Success: True
Iteration 9: Time taken: 0.0020 seconds, Success: True
Iteration 10: Time taken: 0.0050 seconds, Success: True
[2025.08.20 - 16:12:44] Map 2
Iteration 1: Time taken: 0.0305 seconds, Success: True
Iteration 2: Time taken: 0.0180 seconds, Success: True
Iteration 3: Time taken: 0.0250 seconds, Success: True
Iteration 4: Time taken: 0.0140 seconds, Success: True
Iteration 5: Time taken: 0.0295 seconds, Success: True
Iteration 6: Time taken: 0.0190 seconds, Success: True
Iteration 7: Time taken: 0.0240 seconds, Success: True
Iter

In [104]:
def enforce_non_decreasing(arr):
    if not arr:
        return []

    result = [arr[0]]  # 첫 번째 값은 그대로
    for i in range(1, len(arr)):
        if arr[i]['objective'] > result[-1]['objective']:
            result.append(result[-1])   # 이전 값 유지
        else:
            result.append(arr[i])       # 현재 값 유지
    return result

In [119]:
alg_list1 = [basic, expert1, analysis]

In [120]:
[alg['objective'] for alg in alg_list1[1]]

[np.float64(-31.718902660967114),
 np.float64(-35.55809311918709),
 np.float64(-35.65832705434508),
 np.float64(-35.677893170611235),
 np.float64(-35.69658464573582),
 np.float64(-35.586529503601405),
 np.float64(-35.53132893471206),
 np.float64(-35.62448288232131),
 np.float64(-35.7394932468538),
 np.float64(-35.57949504073756)]

In [160]:
basic, expert1, analysis = enforce_non_decreasing(basic), enforce_non_decreasing(expert1), enforce_non_decreasing(analysis)

alg_list = [basic, expert1, analysis]

In [122]:
analysis

[{'operator': 'e2',
  'algorithm_description': 'This hybrid algorithm combines the bidirectional search efficiency of the first dual-tree RRT with the path quality and rewiring optimization of the second algorithm’s RRT*-style rewiring. It grows two trees simultaneously from start and goal, incorporating goal bias sampling for faster convergence and rewiring in each tree for improved path cost. The algorithm alternates growth between the two trees and attempts to connect them via incremental steer steps, ensuring collision-free edges. The Node class includes an `update_parent` method to support clean rewiring, promoting robustness and path smoothness.',
  'planning_mechanism': 'The planner samples random points with goal bias, extends one tree towards the sample, rewires neighbors to optimize cost, then attempts to connect the other tree via a rewiring-enhanced incremental connection procedure. If connection succeeds, paths from both trees are combined into the final path. This bidirec

In [107]:
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go


In [153]:
print(["bottom center"]+["top center"]*7+["bottom center"]*2)

['bottom center', 'top center', 'top center', 'top center', 'top center', 'top center', 'top center', 'top center', 'bottom center', 'bottom center']


In [168]:
fig = go.Figure()

name = ['EoH', 'Expert DB(Ours)', 'Expert+Analysis DB(Ours)']
p_list = []
p_list.append(["top center"]+["bottom center"]+["top center"]*8)
p_list.append(["bottom center"]+["top center"]*9)
p_list.append(["bottom center"]+["top center"]*7+["bottom center"]*2)

for i, alg in enumerate(alg_list):
    fig.add_trace(go.Scatter(
        x=[i+1 for i in range(len(alg))],
        y=[-x['objective'] for x in alg],
        mode="lines+markers+text",
        text=["{:.2f}".format(-x['objective']) for x in alg],           # 표시할 텍스트 값
        textposition=p_list[i],
        name=name[i],
        hovertemplate="Step: %{x}<br>Objective: %{y}<br>Operator: {}"
    ))

fig.update_layout(
    width=800,    # 가로 크기 (px)
    height=500,   # 세로 크기 (px)
)


# 레이아웃
fig.update_layout(
    # title="Objective over Number of Generations",
    xaxis_title="Number of generations",
    yaxis_title="Objective",
    template="plotly_white",
    legend_title="Method"
)
fig.update_layout(
    legend=dict(
        x=0.65,   # 가로 위치 (0=왼쪽, 1=오른쪽)
        y=0.1,  # 세로 위치 (0=아래, 1=위)
        bgcolor="rgba(255,255,255,0.7)",  # 배경 색상 (투명도 조절)
        bordercolor="black",              # 테두리 색상
        borderwidth=1
    )
)
fig.update_layout(
    xaxis=dict(
        showline=True,          # 축 라인 표시
        linewidth=2,            # 축 라인 두께
        linecolor="black",      # 축 라인 색
        showgrid=True,          # 그리드 표시 여부
        gridcolor="lightgray",  # 그리드 색상
        tickmode="linear",      # 눈금 방식 (linear / array / auto)
        dtick=1,                # 눈금 간격 (예: 1씩 증가)
        tickangle=0,            # 눈금 글자 각도
        tickfont=dict(size=12)  # 눈금 글자 크기
    )
)
fig.update_layout(
    yaxis=dict(
        showline=True,          # 축 라인 표시
        linewidth=2,            # 축 라인 두께
        linecolor="black",      # 축 라인 색
        showgrid=True,          # 그리드 표시 여부
        gridcolor="lightgray",  # 그리드 색상
        tickmode="linear",      # 눈금 방식 (linear / array / auto)
        dtick=5,                # 눈금 간격 (예: 1씩 증가)
        tickangle=0,            # 눈금 글자 각도
        tickfont=dict(size=12)  # 눈금 글자 크기
    )
)
fig.show()

### LM TEST

In [1]:
from eoh.llm.interface_LLM import InterfaceLLM
from eoh.utils.getParas import Paras
from eoh.methods.eoh.eoh_evolution import Evolution
from eoh.problems.optimization.classic_benchmark_path_planning.utils.prompts import GetPrompts
import re
paras = Paras()

In [3]:
ee = Evolution("api.openai.com", api_key, "gpt-4.1-mini-2025-04-14", False, None, True, GetPrompts())
illm = InterfaceLLM("api.openai.com", api_key, "gpt-4.1-mini-2025-04-14", False, None, True)

- check LLM API
remote llm api is used ...
- check LLM API
remote llm api is used ...


In [176]:
pc = ee.get_prompt([result_data[3]], 'e1')

In [8]:
gcode = illm.get_response(pc)

In [9]:
print(gcode)

{This algorithm implements a Bidirectional Informed RRT* planner with heuristic-based informed sampling and path smoothing. It grows two trees from start and goal states, focusing samples in an ellipsoidal informed set to concentrate the search on promising regions, improving efficiency and solution quality. It applies rewiring to optimize paths and uses a post-processing smoothing step for path smoothness and shorter paths.}  
(A random sample is drawn within an ellipsoid defined by the current best path cost to bias sampling, then nodes are extended towards samples from both trees. The two trees attempt to connect after each iteration, enabling quick discovery of a path. Rewiring improves local paths iteratively. When a path is found, path smoothing refines it by shortcutting unnecessary waypoints.)

```python
from typing import List, Tuple, NamedTuple, Optional
from math import dist, sqrt, cos, sin, acos
import random

# --- Node class ---
class Node:
    def __init__(self, position

In [11]:
c, m, a = ee._extract_alg(gcode)

In [18]:
response = ee.get_analysis(classic_method[0]['code'], classic_method[5]['code'], 'time')

In [17]:
classic_method[0]

{'algorithm': 'RRT',
 'algorithm_description': 'This algorithm is a sampling-based path planning algorithm that incrementally builds a space-filling tree rooted at the start position by randomly sampling the configuration space and extending the nearest existing node toward the sample. It continues this process until the goal is reached or a maximum number of iterations is exceeded.',
 'planning_mechanism': 'The planner randomly samples free configurations, finds the nearest node in the current tree, attempts to extend toward the sample by a fixed step size, and adds the new node if the move is valid. This repeats until the goal is reached or iteration limit is hit.',
 'code': '\nclass Node:\n    def __init__(self, position, parent=None, cost=0.0):\n        self.position = position\n        self.parent = parent\n        self.cost = cost\n        self.children = []\n        self.valid = True\n\n    def add_child(self, child_node):\n        self.children.append(child_node)\n\n\nclass Pla

In [19]:
print(response)

1. Summary of key changes:  
   - alg1 implements a bidirectional search with two trees grown from start and goal, while alg2 uses a unidirectional single tree.  
   - alg1 includes rewiring to optimize paths and maintain cost-minimizing parent-child relationships; alg2 lacks rewiring and cost optimization beyond simple parent assignment.  
   - alg2 applies goal biasing (goal_sample_rate) to probabilistically sample the goal position, increasing the chances of connecting to the goal.  
   - alg2 uses a smaller step_size (3.0 vs 5.0) potentially providing finer resolution in path expansion.  
   - alg1 maintains and updates children lists and edges extensively during rewiring; alg2 manages simpler parent-child linkage without edge rewiring.

2. Primary contributors to the performance improvement:  
   - Introduction of goal biasing in sampling significantly speeds goal reachability.  
   - Simpler unidirectional search reduces computational overhead compared to managing two trees and r

In [ ]:
print( "Traceback (most recent call last):\n  File \"c:\\workspace\\eoh_path_planning\\eoh\\src\\eoh\\problems\\optimization\\classic_benchmark_path_planning\\run.py\", line 96, in evaluate\n    exec(self.import_string+code_string, planning_module.__dict__)\nTypeError: can only concatenate str (not \"NoneType\") to str\n")

Traceback (most recent call last):
  File "c:\workspace\eoh_path_planning\eoh\src\eoh\problems\optimization\classic_benchmark_path_planning\run.py", line 96, in evaluate
    exec(self.import_string+code_string, planning_module.__dict__)
TypeError: can only concatenate str (not "NoneType") to str



: 

In [8]:
time.strftime('%Y/%m/%d - %H:%M:%S')

'2025/08/12 - 11:22:56'

In [11]:
random.randint(0, 1)


1

In [9]:
import random

In [8]:
inv = 163.66

(986.327 - inv) / 986.327

0.8340712562872151